In [1]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
import json
from pathlib import Path
import pprint

In [3]:
from dotenv import load_dotenv
load_dotenv()

groq_key = os.getenv("groq_key")

data_path = os.getenv("data_path")
data_path = Path(data_path)

In [4]:
from main import main
from query import DataLoader

c:\Users\grays\anaconda3\envs\capstone\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\grays\OneDrive\Documents\GitHub\DataSci210_MedicationDeprescriber\src\extraction.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
c:\Users\grays\anaconda3\envs\capstone\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.

In [5]:
data_loader = DataLoader(data_path)

In [6]:
experiment_name = 'llmsummary_exp_1'

In [7]:
def load_or_create_csv(experiment_name, df):
    try:
        # Attempt to read the CSV file
        curr_experiment = pd.read_csv(data_path / f"{experiment_name}.csv")
        return curr_experiment
    except FileNotFoundError:
        # If the file does not exist, create it from the incoming DataFrame
        df.to_csv(data_path / f"{experiment_name}.csv", index=False)
        return df

In [8]:
labeled_df = data_loader.get_label_df()

In [9]:
labeled_df = load_or_create_csv(experiment_name, labeled_df)

In [10]:
# Check if 'token_usage' column exists, if not create it
if 'token_usage' not in labeled_df.columns:
    labeled_df['token_usage'] = 0  # Initialize with zeros or any default value


for index, row in labeled_df.iterrows():
    if row["token_usage"] > 0:
        print('skip:',index)
        continue
    else:
        encounter_key = row["key"]
        print("label_index:",index," key:",encounter_key)
        (
            ret_recommendation,
            ret_reasoning,
            token_usage,
            search_history_so_far,
            token_count_history,
        ) = main(groq_key=groq_key, data_path=data_path, encounter_key=encounter_key)
        
        print(ret_recommendation)
        labeled_df.at[index, 'ret_recommendation'] = ret_recommendation
        labeled_df.at[index, 'ret_reasoning'] = ret_reasoning
        labeled_df.at[index, 'token_usage'] = token_usage
        labeled_df.at[index, 'search_history_so_far'] = str(search_history_so_far)
        labeled_df.at[index, 'token_count_history'] = str(token_count_history)

label_index: 0  key: D6253A5CE371EA
continue
label_index: 1  key: DFAFED1811B871
stop
label_index: 2  key: D4865B8BBB294E
continue
label_index: 3  key: D00F3A8D5F43B2
deprescribe
label_index: 4  key: DA15CCF42ACF8B
continue
label_index: 5  key: DFB07E6B8F0957
deprescribe
label_index: 6  key: D1BD3665C06499
continue
label_index: 7  key: D068E26FFF9F43
deprescribe
label_index: 8  key: D0E44FD0BBD96F
deprescribe
label_index: 9  key: D17AFAD1D7BB68
deprescribe
label_index: 10  key: DAA6DE9D2EC973
continue
label_index: 11  key: D5A7045ED60A2E
deprescribe
label_index: 12  key: D4BAEF91CA7588
deprescribe
label_index: 13  key: DACBDED2F35C67
deprescribe
label_index: 14  key: DA44DBBF3CCE49
deprescribe
label_index: 15  key: DAABF28BCE02C7
continue
label_index: 16  key: DB1D9E08BBEC87
continue
label_index: 17  key: D6F69E18CCF3AE
deprescribe
label_index: 18  key: D35D002F7350C0
deprescribe
label_index: 19  key: DF655634638393
deprescribe
label_index: 20  key: D8204A1E91DE84
deprescribe
label_ind

In [11]:
labeled_df.to_csv(data_path / f'{experiment_name}.csv', index=False)

In [12]:
labeled_df

,review_by,validated_by,key,reason,recommendation,diagnosis,GS_response,Note,token_usage,ret_recommendation,ret_reasoning,search_history_so_far,token_count_history
0,Ben,NaN,D6253A5CE371EA,example with gastrointestinal bleed should con...,continue,3-Upper GI Bleed,This patient has a history of a gastrointestin...,NaN,9589,continue,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '1', 'explanation': ""Th...","{'diagnosis_source_continue': 592, 'diagnosis_..."
1,Ben,NaN,DFAFED1811B871,example with a documented history of a GI blee...,continue,3-Upper GI Bleed,This patient has a history of a gastrointestin...,NaN,9677,stop,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '1', 'explanation': ""Th...","{'diagnosis_source_continue': 679, 'diagnosis_..."
2,Ben,NaN,D4865B8BBB294E,example with GI bleeding that occurred during ...,continue,3-Upper GI Bleed,GI bleeding developed during hospital stay and...,NaN,10145,continue,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '1', 'explanation': 'Th...","{'diagnosis_source_continue': 797, 'diagnosis_..."
3,Ben,NaN,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI s...,deprescribe,9-Treated GERD,09/20/2023 inpatient nursing note mentions use...,NaN,8197,deprescribe,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 731, 'diagnosis_..."
4,Ben,NaN,DA15CCF42ACF8B,"Barrett's esophagus, continue PPI",continue,0-Barrett's Esophagus,Patient should continue lansoprazole 30mg ever...,NaN,8788,continue,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 617, 'diagnosis_..."
5,Ben,NaN,DFB07E6B8F0957,example with chronic NSAID use and pharmacist ...,continue,1-Chronic NSAID Use,Patient should stop lansoprazole 30mg twice da...,NaN,8385,deprescribe,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 597, 'diagnosis_..."
6,Ben,NaN,D1BD3665C06499,example where pt should be on due to history o...,continue,2-Severe GERD/Esophagitis,Patinet should continue omeprazole 40mg twice ...,NaN,9706,continue,**Summary of Medication Recommendation**\n\nBa...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 660, 'diagnosis_..."
7,Ben,NaN,D068E26FFF9F43,example where pt came in on and no clear reaso...,deprescribe,10-No Diagnosis,Patient came in on lansoprazole 30mg. No clea...,NaN,5695,deprescribe,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 665, 'diagnosis_..."
8,Ben,NaN,D0E44FD0BBD96F,example where pt came in on and no clear reaso...,deprescribe,9-Treated GERD,Patient was admitted on Lansoprazole 30mg dail...,NaN,8982,deprescribe,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 623, 'diagnosis_..."
9,Ben,NaN,D17AFAD1D7BB68,should stop PPI based on Fundoplication being ...,deprescribe,9-Treated GERD,Patient on home esomeprazole 40mg converted to...,breaks API due to JSON error,8932,deprescribe,**Medication Recommendation Summary**\n\nBased...,"[{'diagnosis_boolean': '0', 'explanation': ""Th...","{'diagnosis_source_continue': 610, 'diagnosis_..."


In [28]:
encounter_key = "SyntheticPt5"

In [29]:
(
    final_recommendation,
    final_explanation,
    token_usage,
    search_history_thus_far_list,
    token_count_history,
) = main(groq_key=groq_key, data_path=data_path, encounter_key=encounter_key)

In [34]:
final_recommendation

'deprescribe'

In [35]:
pprint.pprint(final_explanation)

('**Medication Recommendation Summary**\n'
 '\n'
 "Based on the patient's health information, the recommended action for the "
 "patient's current medication is to **deprescribe**. This recommendation is "
 "primarily based on the patient's past medical history of Gastroesophageal "
 'Reflux Disease (GERD) and the ongoing management of GERD symptoms, as noted '
 "in the patient's medical notes history (source: notes). The patient's "
 'current symptoms, including abdominal pain primarily located in the '
 'epigastric region, are consistent with GERD symptoms. However, there is no '
 'direct evidence of mild to moderate esophagitis or esophageal inflammation, '
 "suggesting that the patient's current medication may be unnecessary.")


In [36]:
token_usage

9111

In [37]:
pprint.pprint(search_history_thus_far_list)

[{'associated diagnosis list': ['Barretts Esophagus or esophageal cell changes',
                                'Chronic Non-Steroidal Anti Inflammatory '
                                '(NSAID) use or GI prophylaxis NSAID use',
                                'Severe esophagitis including bleeding '
                                'esophagitis or esophageal ulcer',
                                'History of gastrointestinal bleeding, gastric '
                                'ulcer, upper GI bleed, or peptic ulcer '
                                'hemorrhage'],
  'diagnosis_boolean': '0',
  'explanation': 'The provided information does not include any of the '
                 'specified conditions. The patient has a diagnosis of GERD, '
                 'which may be a reason for PPI use, but it is not one of the '
                 'conditions listed in the question. Therefore, there is no '
                 'evidence of the specified conditions.',
  'recommendation considered': 

In [38]:
pprint.pprint(token_count_history)

{'diagnosis_source_continue': 595,
 'diagnosis_source_deprescribe': 563,
 'diagnosis_source_stop': 691,
 'encounters_source_continue': 724,
 'encounters_source_deprescribe': 652,
 'encounters_source_stop': 711,
 'final_summary': 2215,
 'notes_source_continue': 962,
 'notes_source_deprescribe': 916,
 'notes_source_stop': 1082}
